<a href="https://colab.research.google.com/github/Giftedz/Learn-machine-learning/blob/main/%E5%86%B3%E7%AD%96%E6%A0%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import math
import copy
import matplotlib.pyplot as plt
import matplotlib as mpl
#plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体
#plt.rcParams['axes.unicode_minus'] = False

In [3]:
dataset = pd.read_excel('/Users/zhangtiansheng/Downloads/WaterMelon_3.0.xlsx',index_col=0)
dataset

FileNotFoundError: ignored

In [ ]:
Attributes = dataset.columns #所有属性的名称
m,n = np.shape(dataset)
dataset = np.matrix(dataset)
for i in range(m):
    if dataset[i,n-1]=="是":
        dataset[i,n-1] = "good"
    else:
        dataset[i,n-1] = "bad"

In [ ]:
attributeList = [] # 属性列表，每一个属性的取值，列表中元素是集合
for i in range(n):
    attribute_value = set()
    for j in range(m):
        attribute_value.add(dataset[j,i])
    attributeList.append(attribute_value)

D = np.arange(0,m,1) # 表示每一个样本编号
A = list(np.ones(n))  # 表示每一个属性是否被使用，使用过了标为 -1
A[-1] = -1
EPS = 0.000001

In [ ]:
class Node:
    def __init__(self,title):
        self.title = title
        self.v = 1   # 节点的信息标记
        self.children = []
        self.deep = 0  # 节点深度
        self.ID = -1


In [ ]:
def isSameY(D):
    curY = dataset[D[0],n-1]
    for i in range(1,len(D)):
        if dataset[D[i],n-1] != curY:
            return False
    return True

def isBlankA(A):
    for i in range(n):
        if A[i] > 0:
            return False
    return True

def isSameAinD(D,A): # 判断在D中，是否所有的未使用过的样本属性均相同
    for i in range(n):
        if A[i] > 0:
            for j in range(1,len(D)):
                if not isSameValue(dataset[D[0],i],dataset[D[j],i]):
                    return False
    return True

def isSameValue(v1,v2):
    if type(v1)==float:
        return abs(v1-v2)<EPS
    else: return  v1==v2

In [ ]:
def mostCommonY(D):
    res = dataset[D[0],n-1]
    count = {}
    count[res] = 1
    maxC = 1
    for i in range(1,len(D)):
        curL = dataset[D[i],n-1]
        if curL not in count:
            count[curL] = 1
        else:
            count[curL] += 1
        if count[curL] > maxC:
            maxC = count[curL]
            res = curL
    return res

In [ ]:
def entropyD(D): #信息熵
    count = {}
    for i in D:
        curY = dataset[i,n-1]
        if curY not in count:
            count[curY] = 1
        else:
            count[curY] += 1
    ans = 0
    total = len(D)
    for i in count.keys():
        ans -= count[i]/total*math.log2(count[i]/total)
    return ans

In [ ]:
def gain(D,a):  #信息增益
    if type(dataset[D[0],a])!=str and abs(dataset[D[0],a]-round(dataset[D[0],a])) > EPS:  # 判断若是连续属性，则调用另一个函数
        res, divideV = gainFloat(D,a)
    else:
        count = {}
        for i in D:
            av = dataset[i,a]
            if av not in count:
                count[av] = [i]
            else:
                count[av].append(i)
        res = entropyD(D)
        total = len(D)
        for i in count.keys():
            res -= len(count[i])/total*entropyD(count[i])
        divideV = -1000  # 这个只是随便给的一个值，没有实际意义
    return res, divideV

In [ ]:
def gainFloat(D,a):
    av = set()
    for i in D:
        av.add(dataset[i,a])
    av = list(av)
    av.sort()
    T = []
    for i in range(len(av)-1):
        T.append((av[i]+av[i+1])/2)
    res = entropyD(D)
    ans = 0
    divideV = T[0]
    for i in T:
        right = []
        left = []
        for j in D:
            if dataset[j,a] < i:
                left.append(j)
            else:
                right.append(j)
        temp = res-len(right)/len(D)*entropyD(right)-len(left)/len(D)*entropyD(left)

        if temp > ans:
            ans = temp
            divideV = i
    return ans, divideV

In [ ]:
def treeGenerate(D,A,title):
    node = Node(title)
    
    if isSameY(D):
        node.v = dataset[D[0],n-1]
        return node
    
    if isBlankA(A) or isSameAinD(D,A):
        node.v = mostCommonY(D)
        return node
    
    
    entropy = 0
    floatV = 0
    p = 0
    for i in range(len(A)):      # 循环遍历A,找可以获得最大信息增益的属性
        if(A[i]>0):
            curEntropy,divideV = gain(D,i)
            
            if curEntropy > entropy:
                p = i                     # 存贮属性编号
                entropy = curEntropy
                floatV = divideV
   
    if isSameValue(-1000,floatV):   # 说明是离散属性
        node.v = Attributes[p]+"=?"
        curSet = attributeList[p]
        for i in curSet:
            Dv = []
            for j in D:
                if dataset[j,p]==i:
                    Dv.append(j)
            if len(Dv)==0:
                nextNode = Node(i)
                nextNode.v = mostCommonY(D)
                node.children.append(nextNode)
            else:
                newA = copy.deepcopy(A)  # 注意是深度复制，否则会改变A中的值
                newA[p] = -1
                nextNode = treeGenerate(Dv,newA,i)
                node.children.append(nextNode)
    else:
        node.v = Attributes[p]+"<="+str(floatV)+"?"
        Dright = []
        Dleft = []
        for i in D:
            if dataset[i,p]<=floatV:
                Dleft.append(i)
            else:Dright.append(i)
        node.children.append(treeGenerate(Dright,A,"Yes"))
        node.children.append(treeGenerate(Dleft,A,"No"))
    return node

In [ ]:
def countLeaf(root,deep): #得到叶子数以及树的最大深度
    root.deep = deep
    res = 0
    if root.v=="good" or root.v=="bad":
        res += 1
        return res,deep
    curdeep = deep
    for i in root.children:
        a,b = countLeaf(i,deep+1)
        res += a
        if b>curdeep: curdeep = b
    return res,curdeep

def giveLeafID(root,ID):
    if root.v=="good" or root.v=="bad":
        root.ID = ID
        ID += 1
        return ID
    for i in root.children:
        ID = giveLeafID(i,ID)
    return ID



In [ ]:
#绘制决策树函数
def plotNode(nodeTxt,centerPt,parentPt,nodeType):     # 绘制节点
    plt.annotate(nodeTxt,xy = parentPt,xycoords='axes fraction',xytext=centerPt,
                 textcoords='axes fraction',va="center",ha="center",color='aqua',bbox=nodeType,
                 arrowprops=arrow_args)
def dfsPlot(root):
    if root.ID==-1:          # 说明根节点不是叶子节点
        childrenPx = []
        meanPx = 0
        for i in root.children:
            cur = dfsPlot(i)
            meanPx += cur
            childrenPx.append(cur)
        meanPx = meanPx/len(root.children)
        c = 0
        for i in root.children:
            nodetype = leafNode
            if i.ID<0: nodetype=decisionNode
            plotNode(i.v,(childrenPx[c],0.9-i.deep*0.8/deep),(meanPx,0.9-root.deep*0.8/deep),nodetype)
            plt.text((childrenPx[c]+meanPx)/2,(0.9-i.deep*0.8/deep+0.9-root.deep*0.8/deep)/2,i.title)
            c += 1
        return meanPx
    else:
        return 0.1+root.ID*0.8/(cnt-1)


In [ ]:
#生成决策树
myDecisionTree = treeGenerate(D,A,'root')
cnt,deep = countLeaf(myDecisionTree,0)
giveLeafID(myDecisionTree,0)

5

## 绘制决策树

In [1]:
decisionNode = dict(boxstyle = "round",fc = "black") #fc背景颜色，ec边框颜色
leafNode = dict(boxstyle = "round4",fc="black",ec='red')
arrow_args = dict(arrowstyle = "<-",color='black')
fig = plt.figure(1,facecolor='white')
rootX = dfsPlot(myDecisionTree)
plotNode(myDecisionTree.v,(rootX,0.9),(rootX,0.9),decisionNode)
plt.show()


NameError: ignored

###哈哈哈